In [1]:
import pandas as pd
import requests
import time
import os
from datetime import datetime, timedelta
from typing import Dict, List, Any, Tuple, Optional
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# -----------------------------
# 1) SETTINGS (USER EDITS HERE)
# -----------------------------
BASE_URL = "https://impilo-dhis.mohcc.gov.zw"
username = "tsncube"
password = "Tsncube@2025"

# Add / remove (comment out) programs here anytime
programs = {
    "HTS Register Mobile App": "uwq1KhUSzZ1",
    "ART Register": "ljMn1YcJy83",
    # "OPD Register": "cV5WlX6kYRO",
    # "Delivery Register": "GqLPb8u0rCS",
    # "Malaria Register": "eRHTtRCwopx",
}

# Start date (required) - ddmmyyyy
USER_START_DATE_STR = "01122025"

# End date (optional) - ddmmyyyy
# If you COMMENT OUT the next line, the script will default to TODAY
USER_END_DATE_STR = "31122025"  # <-- comment this line to default to current date

# Reliability / speed knobs
PAGE_SIZE = 1000
REQUEST_TIMEOUT = 120

# Chunking strategy (we will auto-reduce if a chunk fails)
INITIAL_CHUNK_DAYS = 7
MIN_CHUNK_DAYS = 1

# How many times to retry a chunk (before splitting smaller)
CHUNK_RETRIES = 3

# Save summary incrementally
SAVE_AFTER_EACH_CHUNK = True

# -----------------------------
# 2) HELPERS
# -----------------------------
def parse_ddmmyyyy(date_str: str) -> datetime:
    return datetime.strptime(date_str, "%d%m%Y")


def make_session(username: str, password: str) -> requests.Session:
    session = requests.Session()
    session.auth = (username, password)

    retry = Retry(
        total=10,
        connect=10,
        read=10,
        status=10,
        backoff_factor=1.0,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["GET"],
        raise_on_status=False,
        respect_retry_after_header=True,
    )
    adapter = HTTPAdapter(max_retries=retry, pool_connections=20, pool_maxsize=20)
    session.mount("https://", adapter)
    session.mount("http://", adapter)
    session.headers.update({"Accept": "application/json"})
    return session


def dhis2_get_json(session: requests.Session, endpoint: str, params: Dict[str, Any]) -> Dict[str, Any]:
    url = f"{BASE_URL}{endpoint}"
    r = session.get(url, params=params, timeout=REQUEST_TIMEOUT)
    r.raise_for_status()
    return r.json()


def test_connection(session: requests.Session) -> None:
    me = dhis2_get_json(session, "/api/me.json", {"fields": "id,name,username"})
    print(f"✅ Successfully connected to Impilo DHIS2 as: {me.get('name')} ({me.get('username')})")


def build_dataelement_name_map(session: requests.Session, program_uid: str) -> Tuple[Dict[str, str], Dict[str, Optional[str]]]:
    """
    Returns:
      - de_map: dataElement UID -> dataElement name
      - de_optionset_map: dataElement UID -> optionSet UID (or None)
    """
    meta = dhis2_get_json(
        session,
        f"/api/programs/{program_uid}.json",
        {
            "fields": (
                "id,name,"
                "programStages[id,programStageDataElements[dataElement[id,name,code,optionSet[id]]]]"
            )
        },
    )

    de_map: Dict[str, str] = {}
    de_optionset_map: Dict[str, Optional[str]] = {}

    for ps in meta.get("programStages", []):
        for psde in ps.get("programStageDataElements", []):
            de = psde.get("dataElement", {}) or {}
            de_id = de.get("id")
            de_name = de.get("name")

            if de_id and de_name:
                de_map[de_id] = de_name

            os_obj = de.get("optionSet")
            if de_id:
                de_optionset_map[de_id] = os_obj.get("id") if isinstance(os_obj, dict) and os_obj.get("id") else None

    return de_map, de_optionset_map


def build_options_name_map(session: requests.Session, option_set_ids: List[str]) -> Dict[str, str]:
    """
    Fetch ALL options per optionSet using /api/options.json (paging=false).

    Returns map that supports BOTH:
      - option ID -> option name
      - option CODE -> option name
    """
    option_map: Dict[str, str] = {}
    unique_sets = sorted(set([x for x in option_set_ids if x]))

    for os_id in unique_sets:
        page = dhis2_get_json(
            session,
            "/api/options.json",
            {
                "filter": f"optionSet.id:eq:{os_id}",
                "paging": "false",
                "fields": "id,code,name",
            },
        )

        for opt in page.get("options", []) or []:
            opt_id = opt.get("id")
            opt_code = opt.get("code")
            opt_name = opt.get("name")

            if opt_name:
                if opt_id:
                    option_map[opt_id] = opt_name
                if opt_code:
                    option_map[opt_code] = opt_name

    return option_map


def daterange_chunks(start: datetime, end: datetime, chunk_days: int) -> List[Tuple[datetime, datetime]]:
    chunks = []
    cur = start
    while cur <= end:
        chunk_end = min(cur + timedelta(days=chunk_days - 1), end)
        chunks.append((cur, chunk_end))
        cur = chunk_end + timedelta(days=1)
    return chunks


def _filter_events_to_end_date(events: List[Dict[str, Any]], end_date: datetime) -> List[Dict[str, Any]]:
    """
    Keeps only events with eventDate <= end_date (DATE comparison).
    Protects us when we query endDate+1 day to handle exclusive endDate servers.
    """
    end_d = end_date.date()
    kept = []
    for ev in events:
        raw = ev.get("eventDate")
        if not raw:
            continue
        dt = pd.to_datetime(raw, errors="coerce")
        if pd.isna(dt):
            continue
        if dt.date() <= end_d:
            kept.append(ev)
    return kept


def fetch_events_paged(session: requests.Session, program_uid: str, start_date: datetime, end_date: datetime) -> List[Dict[str, Any]]:
    """
    Fetch events between start_date and end_date.

    CRITICAL FIX (NO SKIPPING):
    Some DHIS2 servers treat endDate as EXCLUSIVE.
    We request endDate = (end_date + 1 day), then filter locally back to end_date inclusive.
    """
    all_events: List[Dict[str, Any]] = []
    page = 1

    api_end_date = end_date + timedelta(days=1)  # <-- boundary-day fix

    while True:
        payload = dhis2_get_json(
            session,
            "/api/events.json",
            {
                "program": program_uid,
                "startDate": start_date.strftime("%Y-%m-%d"),
                "endDate": api_end_date.strftime("%Y-%m-%d"),
                "orgUnitMode": "ACCESSIBLE",
                "page": page,
                "pageSize": PAGE_SIZE,
                "totalPages": "true",
                "fields": (
                    "event,eventDate,programStage,orgUnit,orgUnitName,status,lastUpdated,"
                    "geometry,dataValues[dataElement,value]"
                ),
            },
        )

        events = payload.get("events", [])
        if not events:
            break

        # Filter back to <= end_date so we remain accurate
        events = _filter_events_to_end_date(events, end_date)
        all_events.extend(events)

        pager = payload.get("pager", {}) or {}
        page_count = pager.get("pageCount")
        if page_count is not None and page >= int(page_count):
            break

        page += 1

    return all_events


def save_summary_incremental(final_df_indexed: pd.DataFrame, filename: str):
    tmp = filename + ".tmp"
    final_df_indexed.reset_index().to_csv(tmp, index=False)
    os.replace(tmp, filename)


def events_to_dataframe(
    events: List[Dict[str, Any]],
    de_map: Dict[str, str],
    de_optionset_map: Dict[str, Optional[str]],
    option_uid_or_code_to_name: Dict[str, str],
) -> pd.DataFrame:
    records = []
    for event in events:
        row = {
            "Event": event.get("event"),
            "Program stage": event.get("programStage"),
            "Event date": event.get("eventDate"),
            "Organisation unit name": event.get("orgUnitName"),
            "Organisation unit uid": event.get("orgUnit"),
            "Program status": event.get("status"),
            "Last updated on": event.get("lastUpdated"),
            "Longitude": None,
            "Latitude": None,
        }

        geom = event.get("geometry") or {}
        coords = geom.get("coordinates")
        if isinstance(coords, list) and len(coords) >= 2:
            row["Longitude"] = coords[0]
            row["Latitude"] = coords[1]

        for dv in event.get("dataValues", []):
            de_uid = dv.get("dataElement")
            val = dv.get("value")

            # OptionSet translation: option UID -> option NAME OR option CODE -> option NAME
            if de_uid and de_optionset_map.get(de_uid) and isinstance(val, str):
                translated = option_uid_or_code_to_name.get(val)
                if translated is not None:
                    val = translated

            de_name = de_map.get(de_uid, de_uid)
            row[f"{de_name} [{de_uid}]"] = val

        records.append(row)

    df = pd.DataFrame(records)
    df["Event date"] = pd.to_datetime(df["Event date"], errors="coerce")
    df["Formatted Event date"] = df["Event date"].dt.strftime("%d/%m/%Y")
    return df


def chunk_counts(events: List[Dict[str, Any]], full_date_index: pd.Index) -> pd.Series:
    dates = pd.to_datetime(pd.Series([ev.get("eventDate") for ev in events]), errors="coerce").dt.strftime("%d/%m/%Y")
    # Reindex guarantees every date exists (zeros where missing)
    return dates.value_counts().reindex(full_date_index, fill_value=0)


def fetch_chunk_with_no_skip(
    session: requests.Session,
    program_uid: str,
    s: datetime,
    e: datetime,
    program_name: str,
    chunk_no: int,
    total_chunks: int,
    chunk_days: int,
) -> List[Dict[str, Any]]:
    """
    NEVER SKIP: retries chunk; if still failing, splits chunk into smaller chunks until MIN_CHUNK_DAYS.
    """
    chunk_label = f"{program_name} [{chunk_no}/{total_chunks}] {s.date()} to {e.date()}"
    for attempt in range(1, CHUNK_RETRIES + 1):
        try:
            t0 = time.time()
            events = fetch_events_paged(session, program_uid, s, e)
            dt = time.time() - t0
            rate = (len(events) / dt) if dt > 0 else 0
            print(f"✅ {chunk_label} -> {len(events)} events  ({rate:.1f} events/s)")
            return events
        except Exception as ex:
            print(f"⚠️ {chunk_label} attempt {attempt}/{CHUNK_RETRIES} failed: {ex}")

    if chunk_days > MIN_CHUNK_DAYS:
        new_days = max(MIN_CHUNK_DAYS, chunk_days // 2)
        print(f"🔁 Splitting chunk {s.date()} to {e.date()} into smaller chunks ({new_days} days) to avoid timeouts...")

        all_events: List[Dict[str, Any]] = []
        sub_chunks = daterange_chunks(s, e, new_days)
        sub_total = len(sub_chunks)
        for i, (ss, ee) in enumerate(sub_chunks, start=1):
            sub_events = fetch_chunk_with_no_skip(
                session=session,
                program_uid=program_uid,
                s=ss,
                e=ee,
                program_name=program_name,
                chunk_no=i,
                total_chunks=sub_total,
                chunk_days=new_days,
            )
            all_events.extend(sub_events)
        return all_events

    raise RuntimeError(f"❌ Failed permanently even at {MIN_CHUNK_DAYS}-day chunk: {chunk_label}")


# -----------------------------
# 3) MAIN
# -----------------------------
start_date = parse_ddmmyyyy(USER_START_DATE_STR)
try:
    end_date = parse_ddmmyyyy(USER_END_DATE_STR)
except NameError:
    end_date = datetime.today()

if end_date < start_date:
    raise ValueError("End date cannot be earlier than start date.")

date_range = pd.date_range(start=start_date, end=end_date).strftime("%d/%m/%Y")
date_index = pd.Index(date_range, name="Event date")
final_df = pd.DataFrame(index=date_index)

# Make head() show all columns
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 2000)
pd.set_option("display.max_colwidth", 200)

session = make_session(username, password)

# First notification: connection
test_connection(session)
print("➡️ Now fetching event data from DHIS2...")

# Detailed data dict (DataFrames at end)
detailed_data: Dict[str, pd.DataFrame] = {}

today_stamp = datetime.today().strftime("%Y%m%d")
summary_filename = f"Event_Summary_{today_stamp}.csv"

# Process PROGRAM BY PROGRAM (cleaner notifications)
for program_name, program_uid in programs.items():
    print(f"\n==============================")
    print(f"📥 PROGRAM: {program_name}")
    print(f"==============================")

    # Prepare column
    final_df[program_name] = 0

    # Build DE name map once per program (+ optionset info)
    de_map, de_optionset_map = build_dataelement_name_map(session, program_uid)

    # Build option map (option ID->name AND option CODE->name)
    option_uid_or_code_to_name = build_options_name_map(
        session,
        [os_id for os_id in de_optionset_map.values() if os_id]
    )

    # Download with adaptive chunking (no skipping)
    chunk_days = INITIAL_CHUNK_DAYS
    chunks = daterange_chunks(start_date, end_date, chunk_days)
    total_chunks = len(chunks)

    all_events_program: List[Dict[str, Any]] = []

    for idx, (s, e) in enumerate(chunks, start=1):
        pct = int((idx - 1) / total_chunks * 100)
        print(f"🔄 {program_name} progress: {pct}% (starting chunk {idx}/{total_chunks})")

        events = fetch_chunk_with_no_skip(
            session=session,
            program_uid=program_uid,
            s=s,
            e=e,
            program_name=program_name,
            chunk_no=idx,
            total_chunks=total_chunks,
            chunk_days=chunk_days,
        )

        # Update summary counts (reindex ensures missing dates become 0)
        final_df[program_name] = final_df[program_name].astype(int) + chunk_counts(events, date_index).astype(int)

        # Keep for detailed df
        all_events_program.extend(events)

        if SAVE_AFTER_EACH_CHUNK:
            save_summary_incremental(final_df, summary_filename)
            print(f"💾 Summary saved: {summary_filename}")

    print(f"✅ {program_name} completed: 100%")

    # Build detailed dataframe with proper names + optionSet translation
    detailed_data[program_name] = events_to_dataframe(
        all_events_program,
        de_map,
        de_optionset_map,
        option_uid_or_code_to_name
    )

print("\n✅ ALL PROGRAMS COMPLETED.")

# -----------------------------
# 4) OUTPUT SUMMARY  (KEEP YOUR EXACT ENDING BLOCK)
# -----------------------------
# Output summary
final_df = final_df.reset_index()
today_stamp = datetime.today().strftime("%Y%m%d")
final_df.to_csv(f"Event_Summary_{today_stamp}.csv", index=False)

# Example: detailed tables (uncomment what you want to inspect)
# HTS = detailed_data["HTS Register Mobile App"]
# ART = detailed_data["ART Register"]
# HTS.head(5)
# ART.head(5)


✅ Successfully connected to Impilo DHIS2 as: Tsungi Ncube (tsncube)
➡️ Now fetching event data from DHIS2...

📥 PROGRAM: HTS Register Mobile App
🔄 HTS Register Mobile App progress: 0% (starting chunk 1/5)
✅ HTS Register Mobile App [1/5] 2025-12-01 to 2025-12-07 -> 2832 events  (126.7 events/s)
💾 Summary saved: Event_Summary_20260120.csv
🔄 HTS Register Mobile App progress: 20% (starting chunk 2/5)
✅ HTS Register Mobile App [2/5] 2025-12-08 to 2025-12-14 -> 3214 events  (118.1 events/s)
💾 Summary saved: Event_Summary_20260120.csv
🔄 HTS Register Mobile App progress: 40% (starting chunk 3/5)
✅ HTS Register Mobile App [3/5] 2025-12-15 to 2025-12-21 -> 3088 events  (146.2 events/s)
💾 Summary saved: Event_Summary_20260120.csv
🔄 HTS Register Mobile App progress: 60% (starting chunk 4/5)
✅ HTS Register Mobile App [4/5] 2025-12-22 to 2025-12-28 -> 823 events  (179.0 events/s)
💾 Summary saved: Event_Summary_20260120.csv
🔄 HTS Register Mobile App progress: 80% (starting chunk 5/5)
✅ HTS Register M

In [2]:
HTS = detailed_data["HTS Register Mobile App"]
ART = detailed_data["ART Register"]
#HTS.to_csv('HTS3.csv', index=False)
#ART.to_csv('ART.csv', index=False)

In [3]:
HTS.head(2)

,Event,Program stage,Event date,Organisation unit name,Organisation unit uid,Program status,Last updated on,Longitude,Latitude,Received HIV test results [nAzEdbh3vQ7],HTS Type [yeqbOXGikp6],HTS model [y02Q0RxqevG],Verification test done [miqfQlpGVpY],Opted out of HIV testing [b3g84UsUdGV],First test for this pregnancy [bhJ4S2zBnoX],HTS approach [qjW9hzZ42nR],Consent to index testing [QZQqPXo7IXO],Retesting before ART Initiation [vrFTyTSLoi6],Reason for HIV testing [B7T0biEIoEw],HTS Sub model [zleULR7mqop],Sex [Rv5outsLLFN],Pre test information given [LmyMB38Y4y9],Entry point [fmA5lo2aIjx],HTS test result [g7DyuCIRfP9],RTRI test done [Q3bNGFvHO2r],Date of HIV test [mDDDh2EtluD],HTS number [eROaJ2lpVnr],Received post test counselling [w5XPCdOCgMQ],Client profile [wleVSFOIQN0],First test ever in life [rb1xUZCj5vd],Reason for not issuing result [yrg81pO0WKi],Person Id [CQxGQusuvwY],Self identified gender [HL9iYfQ9euv],Birthdate [IPrCK5rDPVz],Age at visit [bwiEnAmO3rK],Hts Test A3 [L6JfJlJ3niZ],Index case HTS number [j5gU1RLT7GJ],Hts Test A1 [Q7dm00hRTKy],Hts Test A2 [sNF0b8YjWg6],Formatted Event date
0,PHB5KF0ClGN,E13JTgutQYr,2025-12-04,ha Matapi Clinic,E7AXNPjenwo,ACTIVE,2026-01-19T18:28:56.394,None,None,YES,Rapid,Facility excluding index case testing,No,No,None,PITC,No,YES,Know HIV Status,Health Facility,Missing,Yes,General VCT Clinic,NEGATIVE,No,2025-12-04,00-0A-25-2025-V-01127,YES,GENERAL POPULATION,NO,Missing,f56909da-74ab-49c1-805e-4f03fdb2b98e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04/12/2025
1,jXDuCDn1C49,E13JTgutQYr,2025-12-04,ha Matapi Clinic,E7AXNPjenwo,ACTIVE,2026-01-19T18:28:55.391,None,None,YES,Rapid,Facility excluding index case testing,No,No,None,CITC,No,NO,PrEP,Health Facility,Female,Yes,General VCT Clinic,NEGATIVE,No,2025-12-04,00-0A-25-2025-V-01123,YES,GENERAL POPULATION,YES,Missing,ef917fcd-f4a5-4207-b4a0-74f463738f6c,FEMALE,1977-12-04,48,NaN,NaN,NaN,NaN,04/12/2025


In [4]:
HTS.columns = HTS.columns.str.replace(r"\s*\[.*?\]", "", regex=True)
ART.columns = ART.columns.str.replace(r"\s*\[.*?\]", "", regex=True)

In [5]:
HTS.to_csv('HTS1.csv', index=False)
ART.to_csv('ART1.csv', index=False)

In [6]:
HTS2 = HTS

In [7]:
HTS2.head(2)

,Event,Program stage,Event date,Organisation unit name,Organisation unit uid,Program status,Last updated on,Longitude,Latitude,Received HIV test results,HTS Type,HTS model,Verification test done,Opted out of HIV testing,First test for this pregnancy,HTS approach,Consent to index testing,Retesting before ART Initiation,Reason for HIV testing,HTS Sub model,Sex,Pre test information given,Entry point,HTS test result,RTRI test done,Date of HIV test,HTS number,Received post test counselling,Client profile,First test ever in life,Reason for not issuing result,Person Id,Self identified gender,Birthdate,Age at visit,Hts Test A3,Index case HTS number,Hts Test A1,Hts Test A2,Formatted Event date
0,PHB5KF0ClGN,E13JTgutQYr,2025-12-04,ha Matapi Clinic,E7AXNPjenwo,ACTIVE,2026-01-19T18:28:56.394,None,None,YES,Rapid,Facility excluding index case testing,No,No,None,PITC,No,YES,Know HIV Status,Health Facility,Missing,Yes,General VCT Clinic,NEGATIVE,No,2025-12-04,00-0A-25-2025-V-01127,YES,GENERAL POPULATION,NO,Missing,f56909da-74ab-49c1-805e-4f03fdb2b98e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04/12/2025
1,jXDuCDn1C49,E13JTgutQYr,2025-12-04,ha Matapi Clinic,E7AXNPjenwo,ACTIVE,2026-01-19T18:28:55.391,None,None,YES,Rapid,Facility excluding index case testing,No,No,None,CITC,No,NO,PrEP,Health Facility,Female,Yes,General VCT Clinic,NEGATIVE,No,2025-12-04,00-0A-25-2025-V-01123,YES,GENERAL POPULATION,YES,Missing,ef917fcd-f4a5-4207-b4a0-74f463738f6c,FEMALE,1977-12-04,48,NaN,NaN,NaN,NaN,04/12/2025


In [8]:
HTS2 = HTS

In [9]:
hts_tst_org = (
    HTS2.groupby("Organisation unit name", dropna=False)
       .size()
       .reset_index(name="HTS_TST")
)

hts_tst_org.loc["Grand Total"] = ["Grand Total", hts_tst_org["HTS_TST"].sum()]

In [10]:
hts_tst_org.head(2)

,Organisation unit name,HTS_TST
0,bu Ceshhar Clinic,71
1,bu Cowdray Park Clinic,395


### HTS_TST

In [11]:
hts_tst = (
    HTS.assign(**{"Event date": pd.to_datetime(HTS["Event date"], errors="coerce").dt.date})
       .groupby("Event date", dropna=False)
       .size()
       .reset_index(name="HTS_TST")
)

hts_tst.loc["Grand Total"] = ["Grand Total", hts_tst["HTS_TST"].sum()]


In [12]:
hts_tst.head(2)

,Event date,HTS_TST
0,2025-12-01,587
1,2025-12-02,606


In [13]:
hts_tst.head(2)

,Event date,HTS_TST
0,2025-12-01,587
1,2025-12-02,606


### BY Facility

In [14]:
hts_pos_org = (
    HTS2.query("`HTS test result` == 'POSITIVE'")
        .groupby("Organisation unit name", dropna=False)
        .size()
        .reset_index(name="HTS_POS")
)

hts_pos_org.loc["Grand Total"] = ["Grand Total", hts_pos_org["HTS_POS"].sum()]


In [15]:
hts_pos_org.head(4)

,Organisation unit name,HTS_POS
0,bu Ceshhar Clinic,4
1,bu Cowdray Park Clinic,24
2,bu Cowdray Park Health Center,12
3,bu Dr Shennan Clinic,1


### HTS_POS

In [16]:
hts_pos = (
    HTS.assign(**{"Event date": pd.to_datetime(HTS["Event date"], errors="coerce").dt.date})
       .query("`HTS test result` == 'POSITIVE'")
       .groupby("Event date", dropna=False)
       .size()
       .reset_index(name="HTS_POS")
)

hts_pos.loc["Grand Total"] = ["Grand Total", hts_pos["HTS_POS"].sum()]

In [17]:
hts_pos.head(7)

,Event date,HTS_POS
0,2025-12-01,39
1,2025-12-02,43
2,2025-12-03,35
3,2025-12-04,28
4,2025-12-05,31
5,2025-12-06,1
6,2025-12-08,41


### TX_NEW

In [18]:
ART.head(3)

,Event,Program stage,Event date,Organisation unit name,Organisation unit uid,Program status,Last updated on,Longitude,Latitude,Crypto meningitis Screening,TB Investigated,Date Initiated,Art Initiation Category,Weight,Age,CD4 Count,Cevical cancer screening,Sex,Eligible for TPT,Consent to Follow Up,WHO Stage,Visit type,Date of HIV test,ARV Status,Functional status,Person Id,ART Regimen,ART number,Height,Ophan Status,Formatted Event date
0,G58bhZqyihs,rnBlG0WUNp8,2025-12-04,ha Mbare PolyClinic,G6bGAnRpVN3,ACTIVE,2026-01-20T10:20:56.811,None,None,NO,NO,2025-12-04,NEWLY_INITIATED_ON_ART,NaN,25.0,0.0,NO,Female,Missing,Missing,One,Present self,2025-12-04,START ARV,WORK/SCHOOL GOING,9fbbed2f-be69-4182-83d7-db0220aa82ed,1i TDF+3TC+DTG,00-0A-27-2025-A-01322,NaN,NaN,04/12/2025
1,A8YxEvuqtnT,rnBlG0WUNp8,2025-12-04,ha Mbare PolyClinic,G6bGAnRpVN3,ACTIVE,2026-01-20T10:20:55.426,None,None,NO,NO,2025-12-04,NEWLY_INITIATED_ON_ART,70.0,25.0,0.0,NO,Female,Missing,Missing,One,Present self,2025-12-04,START ARV,WORK/SCHOOL GOING,54cbd22a-e71a-4536-85bd-c8c24275c34e,1i TDF+3TC+DTG,00-0A-27-2025-A-01322,NaN,NaN,04/12/2025
2,qbnCvNPoiLZ,rnBlG0WUNp8,2025-12-04,ha Mbare PolyClinic,G6bGAnRpVN3,ACTIVE,2026-01-20T10:20:52.364,None,None,NO,NO,2025-12-04,NEWLY_INITIATED_ON_ART,NaN,29.0,0,NO,Female,YES,Missing,One,Present self,2025-12-04,START ARV,WORK/SCHOOL GOING,02a76a29-c869-48ca-9171-a474edbc95d8,1i TDF+3TC+DTG,00-0A-27-2025-A-01323,NaN,NaN,04/12/2025


In [19]:
ART2 = ART

In [20]:
tx_new_org = (
    ART2.groupby("Organisation unit name", dropna=False)
       .size()
       .reset_index(name="TX_NEW")
)

tx_new_org.loc["Grand Total"] = ["Grand Total", tx_new_org["TX_NEW"].sum()]

In [21]:
tx_new_org.head(3)

,Organisation unit name,TX_NEW
0,bu Ceshhar Clinic,5
1,bu Cowdray Park Clinic,175
2,bu Dr Shennan Clinic,46


In [22]:
tx_new = (
    ART.assign(**{"Event date": pd.to_datetime(ART["Event date"], errors="coerce").dt.date})
       .groupby("Event date", dropna=False)
       .size()
       .reset_index(name="TX_NEW")
)

tx_new.loc["Grand Total"] = ["Grand Total", tx_new["TX_NEW"].sum()]

In [23]:
tx_new.head(4)

,Event date,TX_NEW
0,2025-12-01,252
1,2025-12-02,299
2,2025-12-03,277
3,2025-12-04,264


In [24]:
final_org_df = (
    hts_tst_org.query("`Organisation unit name` != 'Grand Total'")
        .merge(hts_pos_org.query("`Organisation unit name` != 'Grand Total'"),
               on="Organisation unit name", how="left")
        .merge(tx_new_org.query("`Organisation unit name` != 'Grand Total'"),
               on="Organisation unit name", how="left")
        .fillna(0)
)

# ensure clean integers
num_cols = final_org_df.columns.drop("Organisation unit name")
final_org_df[num_cols] = final_org_df[num_cols].astype(int)


In [25]:
final_org_df.head(4)

,Organisation unit name,HTS_TST,HTS_POS,TX_NEW
0,bu Ceshhar Clinic,71,4,5
1,bu Cowdray Park Clinic,395,24,175
2,bu Cowdray Park Health Center,147,12,0
3,bu Dr Shennan Clinic,96,1,46


In [31]:
final_org_df.to_csv('FINALLL.csv', index=False)

In [30]:
final_org_df

,Organisation unit name,HTS_TST,HTS_POS,TX_NEW
0,bu Ceshhar Clinic,71,4,5
1,bu Cowdray Park Clinic,395,24,175
2,bu Cowdray Park Health Center,147,12,0
3,bu Dr Shennan Clinic,96,1,46
4,bu EF Watson Clinic,126,3,71
...,...,...,...,...
87,mv Rujeko Clinic,141,2,0
88,mv Runyararo Clinic,302,7,9
89,mv Shonganiso Clinic,18,1,7
90,mv Zivuku Clinic,60,2,16


### Combined by Dates

In [27]:
final_df = (
    hts_tst
        .merge(hts_pos, on="Event date", how="left")
        .merge(tx_new, on="Event date", how="left")
        .fillna(0)
        .astype({"HTS_TST": int, "HTS_POS": int, "TX_NEW": int})
)

In [28]:
final_df.head(4)

,Event date,HTS_TST,HTS_POS,TX_NEW
0,2025-12-01,587,39,252
1,2025-12-02,606,43,299
2,2025-12-03,524,35,277
3,2025-12-04,617,28,264


In [29]:
final_df.to_csv('FINALLL by dates.csv', index=False)